In [1]:
import torch
from torch.utils import data
import random
import sklearn
from sklearn.model_selection import train_test_split, StratifiedKFold
from collections import Counter
import pandas as pd
import numpy as np
import scipy
from tqdm import trange
from tqdm import tqdm
from datetime import datetime
import sys
import os
import seaborn as sns
from matplotlib import pyplot as plt
from joblib import Parallel, delayed, dump, load
from matplotlib import pyplot as plt
from sparse_vector.sparse_vector import SparseVector
from scipy.signal import convolve2d, convolve
import time
from torch import nn
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score, f1_score
from IPython.display import clear_output
import warnings
warnings.filterwarnings("ignore")
from dna_tokenizer import DNATokenizer, seq2kmer
from transformers import BertModel, BertConfig, BertForTokenClassification
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score

2023-04-20 13:11:57.971526: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-20 13:11:58.351685: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-04-20 13:12:00.554467: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-04-20 13:12:00.554634: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such 

In [1]:
chroms = [f'chr{i}' for i in list(range(1, 20)) + ['X', 'Y']]
DNA = {chrom:load(f'../data/mm9_dna/sparse/{chrom}.pkl') for chrom in tqdm(chroms)}

NameError: name 'tqdm' is not defined

In [3]:
G4_kouzine = {}
for chrom in DNA:
    G4_kouzine[chrom] = np.zeros(len(DNA[chrom]), dtype = bool)
    
    
with open("actB_ssDNA_enriched_Quadruplex.bed")as f:
    for idx, line in enumerate(f):
        if idx>0:
            chrom, start, end, _ , _ , _ = line.strip().split()
            if chrom in G4_kouzine:
                G4_kouzine[chrom][int(start):int(end)] = 1

dump(G4_kouzine, 'G4_mm9_kouzine.pkl')

['G4_mm9_kouzine.pkl']

In [4]:
G4 = load('G4_mm9_kouzine.pkl')

In [5]:
width = 512

np.random.seed(10)

ints_in = []
ints_out = []


for chrm in chroms:
    for st in trange(0, G4[chrm].shape[0] - width, width):
        interval = [st, min(st + width, G4[chrm].shape[0])]
        if G4[chrm][interval[0]: interval[1]].any():
            ints_in.append([chrm, int(interval[0]), int(interval[1]), 1])
        else:
            ints_out.append([chrm, int(interval[0]), int(interval[1]), 0])
                
print(len(ints_in))
print(len(ints_out))

ints_in_full = ints_in
ints_out_full = ints_out

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 31059/31059 [00:00<00:00, 87007.29it/s]

17830
5167504


In [6]:
ints_in = ints_in_full
ints_out = [ints_out_full[i] for i in np.random.choice(range(len(ints_out_full)), 
                                                    size=len(ints_in) * 2, replace=False)]
print(len(ints_in))
print(len(ints_out))

17830
35660


In [7]:
equalized = ints_in + ints_out

In [8]:
divisions = list(StratifiedKFold(5, shuffle=True, 
                                 random_state=42).split(equalized, [f"{elem[3]}_{elem[0]}"
                                         for i, elem 
                                         in enumerate(equalized)]))

In [9]:
dump([equalized, divisions], 'mm_divisions_kouzine_g4.pkl', 3)

['mm_divisions_kouzine_g4.pkl']

In [10]:
def seq2kmer(seq, k):
    """
    Convert original sequence to kmers
    
    Arguments:
    seq -- str, original sequence.
    k -- int, kmer of length k specified.
    
    Returns:
    kmers -- str, kmers separated by space
    """
    kmer = [seq[x:x+k] for x in range(len(seq)+1-k)]
    kmers = " ".join(kmer)
    return kmers

In [11]:
class Dataset(data.Dataset):
    def __init__(self, chroms, dna_source, 
                 labels_source, intervals, tokenizer):

        self.chroms = chroms
        self.dna_source = dna_source
        self.labels_source = labels_source
        self.intervals = intervals
        self.tokenizer = tokenizer
        
        
    def __len__(self):
        return len(self.intervals)
    
    def __getitem__(self, index):
        interval = self.intervals[index]
        chrom = interval[0]
        begin = int(interval[1])
        end = int(interval[2])
        ll = list(self.dna_source[chrom][begin:end].upper())
        y = self.labels_source[interval[0]][interval[1]: interval[2]]        
                
        k_mers = seq2kmer(self.dna_source[chrom][begin:end+5].upper(),6)
        encoded_k_mers = self.tokenizer.encode_plus(k_mers, add_special_tokens=False, max_length=512)["input_ids"]

        return torch.LongTensor(encoded_k_mers), torch.Tensor(y).long(), (chrom, begin, end)

In [12]:
for MODEL_NUMBER in range(5):

    train_inds, test_inds = divisions[MODEL_NUMBER]
    train_intervals, test_intervals = [equalized[i] for i in train_inds], [equalized[i] for i in test_inds]

    random.shuffle(train_intervals)
    random.shuffle(test_intervals)
    
    train_dataset = Dataset(chroms, DNA, G4, train_intervals, 
                        DNATokenizer.from_pretrained('6-new-12w-0/', add_special_tokens=False))

    test_dataset = Dataset(chroms, DNA, G4, test_intervals,
                          DNATokenizer.from_pretrained('6-new-12w-0/', add_special_tokens=False))
    
    dump((train_dataset, test_dataset), f'ds_w_seq_mm_fold{MODEL_NUMBER}_kouzine_g4.pkl')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'DNATokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'DNATokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'DNATokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected 

In [14]:
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    
    for idx, batch in enumerate(training_loader):
        input_ids, labels, intervals = batch            
        input_ids = input_ids.to(device)
        labels = labels.to(device)
        #print(model.device, input_ids.device, labels.device)
        outputs = model(input_ids = input_ids, labels = labels)        
        #print(outputs)
        loss, tr_logits = outputs['loss'], outputs['logits']
        #print(model(input_ids=ids, attention_mask=mask, labels=labels))
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=0.1
        )
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

In [15]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels, eval_scores = [], [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            input_ids, labels, intervals = batch            
            input_ids = input_ids.to(device)
            labels = labels.to(device)
            outputs = model(input_ids = input_ids, labels = labels)
            loss, eval_logits = outputs['loss'], outputs['logits']            
            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
              
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            flattened_scores = active_logits[:,1] - active_logits[:,0]
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            eval_scores.extend(flattened_scores)
            
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    labels = [id.item() for id in eval_labels]
    predictions = [id.item() for id in eval_preds]
    scores = [id.item() for id in eval_scores]
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions, scores

In [16]:
device = 3
lr = 1e-5
EPOCHS = 10

dir_to_pretrained_model = "6-new-12w-0/"
config = BertConfig.from_pretrained('https://raw.githubusercontent.com/jerryji1993/DNABERT/master/src/transformers/dnabert-config/bert-config-6/config.json')
config.classifier_droput = 0.5
tokenizer = DNATokenizer.from_pretrained('6-new-12w-0/')

for MODEL_NUMBER in range(5):
    train_dataset, test_dataset = load(f'ds_w_seq_mm_fold{MODEL_NUMBER}_kouzine_g4.pkl')
    training_loader = DataLoader(train_dataset, batch_size=24, num_workers = 2)
    testing_loader = DataLoader(test_dataset, batch_size=16, num_workers = 2)


    model = BertForTokenClassification.from_pretrained(dir_to_pretrained_model, config=config)
    model.to(device)
    
    optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=lr, steps_per_epoch=len(training_loader), epochs=EPOCHS)

    for epoch in range(EPOCHS):
        print(f"Training fold {MODEL_NUMBER} epoch: {epoch + 1}")
        train(epoch)
        labels, predictions, scores = valid(model, testing_loader)
        print(f'Fold {MODEL_NUMBER} validation ROC-AUC: ', roc_auc_score(labels, scores))

    print(sklearn.metrics.classification_report(labels, predictions))
    model.save_pretrained(f'dnabert_mm_fold_{MODEL_NUMBER}_kouzine_g4')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'DNATokenizer'.
Some weights of the model checkpoint at 6-new-12w-0/ were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint

Training fold 0 epoch: 1


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Training loss epoch: 0.2475347226612015
Training accuracy epoch: 0.8627318537197374


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


...

Validation Loss: 0.018039187295130262
Validation Accuracy: 0.9922015134529147
Fold 4 validation ROC-AUC:  0.9979270275539248
Training fold 4 epoch: 10


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Training loss epoch: 0.012410589040457352
Training accuracy epoch: 0.9948662005310579


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Validation Loss: 0.018113980178754485
Validation Accuracy: 0.9922539909146114
Fold 4 validation ROC-AUC:  0.9979252462023559
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   5347178
           1       0.82      0.86      0.84    130198

    accuracy                           0.99   5477376
   macro avg       0.91      0.93      0.92   5477376
weighted avg       0.99      0.99      0.99   5477376

